In [2]:
import numpy as np
import pandas as pd

# To process embeddings
import tensorflow as tf
import tensorflow_hub as hub

# To silence warnings from TensorFlow
import os
import logging
import warnings;
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)

# To load saved embeddings
import joblib

# To create webapp
import psutil


#######################################################################################
                            # Load functions
#######################################################################################

def load_embeddings():
    # Path to USE
    embed = hub.load('/media/einhard/Seagate Expansion Drive/3380_data/data/tensorflow_hub/universal-sentence-encoder_4')

    # Load pre-trained sentence arrays
    sentence_array = joblib.load('/media/einhard/Seagate Expansion Drive/3380_data/data/Models/reviewEmbeddings.pkl')
    descriptions_array = joblib.load('/media/einhard/Seagate Expansion Drive/3380_data/data/Models/descriptionEmbeddings.pkl')
    return embed, sentence_array, descriptions_array

def data_loader(datapath, books_file, reviews_file):
    books = pd.read_csv(datapath + books_file).drop('Unnamed: 0', axis=1).fillna('')
    reviews = pd.read_csv(datapath + reviews_file).drop('Unnamed: 0', axis=1)
    return books, reviews


                    # Return recommendations based on reviews
def find_reviews(query,reviews, n_results=5):
    # Create vector from query and compare with global embedding
    sentence = [query]
    sentence_vector = np.array(embed(sentence))
    inner_product = np.inner(sentence_vector, sentence_array)[0]

    # Find sentences with highest inner products
    top_n_sentences = pd.Series(inner_product).nlargest(n_results+1)
    top_n_indices = top_n_sentences.index.tolist()
    top_n_list = list(reviews.review_text.iloc[top_n_indices][1:])

    #print(f'Input sentence: "{query}"\n')
    #print(f'{n_results} most semantically similar reviews: \n\n')
    #print(*top_n_list, sep='\n\n')
    #print(top_n_indices)
    return top_n_indices

def find_books(query, reviews, books, n_results=5):
    top_n_indices = find_reviews(query, reviews, n_results)
    return books[books.book_id.isin(reviews.iloc[top_n_indices].book_id.tolist())][['title', 'name','description', 'weighted_score', 'book_id']].fillna('').reset_index().drop('index', axis=1)


                    # Return recommendations based on descriptions

def find_description(query, books, n_results=10):
    # Create vector from query and compare with global embedding
    sentence = [query]
    sentence_vector = np.array(embed(sentence))
    inner_product = np.inner(sentence_vector, descriptions_array)[0]

    # Find sentences with highest inner products
    top_n_sentences = pd.Series(inner_product).nlargest(n_results)
    top_n_indices = top_n_sentences.index.tolist()
    top_n_list = list(books.description.iloc[top_n_indices][1:])

    #print(f'Input sentence: "{query}"\n')
    #print(f'{n_results} most semantically similar book descriptions: \n\n')
    #print(*top_n_list, sep='\n\n')
    return top_n_indices

def find_books_description(query, reviews, books):
    top_n_indices = find_description(query)
    return books[books.book_id.isin(books.iloc[top_n_indices].book_id.tolist())][['title', 'name','description', 'weighted_score', 'book_id']].fillna('')

def show_recommendations(query, reviews, books, n_results=5):
    top_n_indices = find_reviews(query, reviews, n_results)
    book_recommends = find_books(query, reviews, books, n_results)
    book_recommends['for_url'] = book_recommends['book_id'].astype(str) + '.' + book_recommends['title'].replace(r'\(.*$', '', regex = True)
    return top_n_indices, book_recommends



#######################################################################################
                            # Load variables and data
#######################################################################################

# Path to books and reviews DataFrames
datapath = '/media/einhard/Seagate Expansion Drive/3380_data/data/Filtered books/'

# Books and reviews file names and loading
books_file = 'clean_filtered_books.csv'
reviews_file = 'clean_filtered_reviews.csv'
reviewClusters = pd.read_csv('/media/einhard/Seagate Expansion Drive/3380_data/data/Processed/reviews_for_cluster.csv').drop('Unnamed: 0', axis=1)

# Base URL for goodreads
goodreadsURL = 'https://www.goodreads.com/book/show/'

books, reviews = data_loader(datapath, books_file, reviews_file)
embed, sentence_array, descriptions_array = load_embeddings()


#######################################################################################
                                # Web App
#######################################################################################

FileNotFoundError: [Errno 2] No such file or directory: '/media/einhard/Seagate Expansion Drive/3380_data/data/Processed/reviews_for_cluster.csv'

In [1]:
import re
from sklearn.cluster import KMeans

def embedSentences(book_title):
    sentences = reviews_for_cluster[reviews_for_cluster.book_id.isin(books[books.title.isin([book_title])].book_id.tolist())]['review_text']
    sentence_vectors = embed(sentences)
    return sentences, sentence_vectors

def findClusters(sentences, sentence_vectors, book_title, k, n_results):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(sentence_vectors)
    for i in range(k):
        centre = kmeans.cluster_centers_[i]
        ips = np.inner(centre, sentence_vectors)
        idx = pd.Series(ips).nlargest(n_results).index
        clusteredSentences = list(sentences.iloc[idx])

        #'---'
        #st.write(f'**Cluster #{i+1} reviews:** \n\n*Book title: {book_title}*')
        #for sent in clusteredSentences:
        #    st.write(sent)
        print(f'**Cluster #{i+1} reviews:** \n\n*Book title: {book_title}*')
        for sent in clusteredSentences:
            print(sent)

In [2]:
books[books.title.isin(['Gone Girl'])].book_id

7080    19288043
Name: book_id, dtype: int64

In [11]:
bookID = 19288043
review_max_len = 50
input_book_id = books[books.book_id.isin(bookID)].book_id.tolist()
input_sentences = reviewClusters[reviewClusters.book_id.isin(input_book_id)].review_text
input_sentences = input_sentences[input_sentences.str.len() <= review_max_len]

input_vectors = embed(input_sentences)

#sentences = reviewsClusters[reviewsClusters.book_id.isin(books[books.title.isin([book_title])]#.book_id.tolist())]['review_text']
#sentences[sentences.str.len() < 50]

In [16]:
embeddedReviewClusters = embed(reviewClusters.review_text)

ResourceExhaustedError:  OOM when allocating tensor with shape[83239685,320] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node StatefulPartitionedCall/StatefulPartitionedCall/EncoderDNN/EmbeddingLookup/EmbeddingLookupUnique/GatherV2}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_restored_function_body_5647]

Function call stack:
restored_function_body


In [13]:
def embedAuthorSentences(book_title, review_max_len):
    book_names = books[books.name == book_title].book_id.tolist()
    sentences = reviews_for_cluster[reviews_for_cluster.book_id.isin(book_names)]['review_text']
    sentences = sentences[sentences.str.len() <= review_max_len]
    sentence_vectors = embed(sentences)
    return sentences, sentence_vectors

embedAuthorSentences(book_title, 50)

(9936             Why did I put off reading this for so long!
 76948           One of the best science fiction novels ever.
 94460                      The Godfather of science fiction!
 106924                       Holy crap this book is awesome!
 131745                      One of the best books i've read.
                                  ...                        
 1291104         an interesting scifi story superbly written.
 1328807            I did enjoy this, more so than the movie.
 1359573           Read the Jacqui's Room Notes on Dune here.
 1380716    This continues to be my favorite book of all t...
 1386570                      I enjoyed the whole Dune series
 Name: review_text, Length: 92, dtype: object,
 <tf.Tensor: shape=(92, 512), dtype=float32, numpy=
 array([[ 0.00718922, -0.01291069, -0.00347207, ...,  0.00870479,
          0.03012627, -0.00029959],
        [-0.01894443,  0.01617449, -0.02430125, ..., -0.04384046,
          0.0977076 ,  0.04950066],
        [ 0.004

In [12]:
reviews_for_cluster[reviews_for_cluster.book_id.isin(books[books.title.isin([book_title])].book_id.tolist())]['review_text']

Series([], Name: review_text, dtype: object)

In [9]:
books[books.book_id.isin([104])]

,title,description,book_id,weighted_score,name
3213,"Dune (Dune Chronicles, #1)",Here is the novel that will be forever conside...,104,4.195643,Frank Herbert


In [14]:
author_name = books[books.name.str.contains('Frank Herbert', case=False)].name.tolist()

In [21]:
for idx, authorName in enumerate(author_name):
    info = books[books.name == authorName]
    print(info.title.tolist()[idx])
    print(info.name.tolist()[idx])

Dune (Dune Chronicles #1)
Frank Herbert
Dune
Frank Herbert
Dune (Dune Chronicles, #1)
Frank Herbert
Chapterhouse: Dune (Dune Chronicles #6)
Frank Herbert


In [16]:
books.name.isna().sum()

23

In [12]:
reviews

,book_id,review_text
0,831635,Recommended by our pediatrician
1,27539,6 of my smartest friends rated this 4 or highe...
2,259028,"If steve recommends it, it must be good!"
3,5558,elizabeth got this for free at work... anyone ...
4,1202,I loved the abortion cured crime theory.
...,...,...
429621,12067,Lots of fun. I'll have to try some more Terry ...
429622,5907,What can I say about Tolkien? Just read his wo...
429623,47956,This book is an exhilarating book of magic and...
429624,5139,"Really enjoyed this book. Crazy, fast paced fa..."


In [13]:
reviewClusters

,book_id,review_text
0,831635,Recommended by our pediatrician
1,27539,6 of my smartest friends rated this 4 or highe...
2,54499,A great finish to the D'Artagnan series - defi...
3,77566,Seven amazing stories. Each one you think can'...
4,361459,Highly recommended if you are curious about Ma...
...,...,...
1392328,170210,I loved this book. The author's politics are a...
1392329,140082,I loved this book. The author's politics are a...
1392330,110494,I loved this book. The author's politics are a...
1392331,301082,I loved this book. The author's politics are a...


# Creating title search

In [151]:
books = books[['title','description', 'book_id', 'weighted_score','name']]
books.fillna('', inplace=True)

In [154]:
books.to_csv(datapath+'clean_filtered_books.csv')

In [161]:
reviews = reviews[['book_id','review_text']]
reviews.to_csv(datapath+'clean_filtered_reviews.csv')

In [157]:
reviews

,user_id,book_id,review_id,rating,review_text,date_added,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,831635,549bc8e5f48d65e981cf2abb3ff7ee6f,0,Recommended by our pediatrician,Mon May 16 12:29:47 -0700 2011,0,12
1,8842281e1d1347389f2ab93d60773d4d,27539,b45353478a136de0a3e93077d47b1d6b,0,6 of my smartest friends rated this 4 or highe...,Sun Nov 15 20:28:29 -0800 2009,0,2
2,8842281e1d1347389f2ab93d60773d4d,259028,fb4acc8a30bac6bf1414a03303d43c26,0,"If steve recommends it, it must be good!",Thu Jan 18 11:09:48 -0800 2007,2,2
3,8842281e1d1347389f2ab93d60773d4d,5558,3c01b327de615dc96910130797da1224,0,elizabeth got this for free at work... anyone ...,Thu Dec 07 16:43:16 -0800 2006,0,0
4,8842281e1d1347389f2ab93d60773d4d,1202,8458284a7dcf85d90b3c55bd6b4523d4,4,I loved the abortion cured crime theory.,Mon Nov 13 12:31:32 -0800 2006,6,2
...,...,...,...,...,...,...,...,...
429621,349751201c57bc3b652590c6c90d894c,12067,cf4da327f4a32c6325eca4f8c52ff11c,4,Lots of fun. I'll have to try some more Terry ...,Wed Sep 10 20:54:42 -0700 2008,0,0
429622,349751201c57bc3b652590c6c90d894c,5907,ac0e9d4b8b2ab0144402c22d90ba1df3,5,What can I say about Tolkien? Just read his wo...,Wed Jul 30 18:50:48 -0700 2008,0,0
429623,621c5857e423b0f2283a22604d1796a0,47956,50b83b9ea55af86f2561fa35ba1ff483,5,This book is an exhilarating book of magic and...,Wed Jul 30 20:59:21 -0700 2008,0,0
429624,2c2733058db98a841851789c132e0334,5139,200307bd0e38a59982a77a89157771d9,4,"Really enjoyed this book. Crazy, fast paced fa...",Mon Mar 03 04:22:30 -0800 2014,0,0


In [141]:
books.name.isna().sum()

23

In [110]:
sentence = 'title: Century'

In [9]:
book_title = books[books.title.str.match(sentence, case=False)].title.tolist()
#for i in range(len(book_title))
#    sentences, sentence_vectors = embedSentences(book_title[i])

In [25]:
test = re.search(r'\"[\s\S]*\"', sentence)

In [29]:
book_title = books[books.title.contains(sentence, case=False)].title.tolist()
book_title

AttributeError: 'StringMethods' object has no attribute 'search'

In [111]:
sentence = 'title: Wolf Children: Ame & Yuki'
sentence = sentence.replace('title: ', '')
book_title = books[books.title.isin([sentence])].title.tolist()[0]

In [113]:
books[books.name == 'Frank Herbert']

,title,author,average_rating,ratings_count,text_reviews_count,description,shelf_1,book_id,weighted_score,name
3023,Dune (Dune Chronicles #1),58.0,4.20,494553.0,9982.0,This book was mistakenly published under .\nS...,currently-reading,234225,4.199971,Frank Herbert
3154,Dune,58.0,4.20,8645.0,750.0,Librarian's note: There are Alternate Cover Ed...,currently-reading,53732,4.198359,Frank Herbert
3213,"Dune (Dune Chronicles, #1)",58.0,4.20,3224.0,360.0,Here is the novel that will be forever conside...,currently-reading,104,4.195643,Frank Herbert
10352,Chapterhouse: Dune (Dune Chronicles #6),58.0,3.89,35448.0,476.0,"The desert planet Arrakis, called Dune, has be...",sci-fi,105,3.890035,Frank Herbert


In [114]:
author_name = books[books.name.str.match('Frank Herbert')].name.tolist()

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [123]:
books.name.str.contains('Frank Herbert')
books[books.name.str.contains('Frank Herbert')].name.tolist()

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [127]:
books[books.name.str.contains('Frank Herbert').dropna()]

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [126]:
books.isna().sum()

title                    0
author                  23
average_rating           0
ratings_count            0
text_reviews_count       0
description            258
shelf_1               1863
book_id                  0
weighted_score           0
name                    23
dtype: int64

In [83]:
sentence = 'title: Children'
re.match(r'title: ', sentence)
sentence = sentence.replace('title: ', '')
book_title = books[books.title.str.match(sentence)].title.tolist()
for idx, bookTitle in enumerate(book_title):
    info = books[books.title == bookTitle]

In [100]:
'.'.join([info.book_id.astype(str).tolist()[0], info.title.replace(r'\(.*$', '', regex = True).tolist()[0]])

'16130324.Children of the Jacaranda Tree'

In [103]:
sentence

'Children'

In [106]:
books[books.title.str.findall(sentence)].title.tolist()

TypeError: unhashable type: 'list'

In [109]:
books.title.str.extract(sentence)

ValueError: pattern contains no capture groups

In [65]:
book_title = books[books.title.isin([sentence])].title.tolist()
for idx, bookTitle in enumerate(book_title):
    info = books[books.title == bookTitle]
    print(info)

                         title     author  average_rating  ratings_count  \
404  Wolf Children: Ame & Yuki  5049973.0            4.44         2431.0   

     text_reviews_count                                        description  \
404               351.0  When Hana falls in love with a young interlope...   

    shelf_1   book_id  weighted_score           name  
404   manga  18170149        4.429414  Mamoru Hosoda  


In [51]:
sentencea

'Wolf Children: Ame & Yuki'

In [47]:
print(re.match(r'title:', sentence))

<re.Match object; span=(0, 6), match='title:'>


In [26]:
if test:
    print('YAS')

In [10]:
query = 'The Tournament at Gorlan'

def show_books(sentence):
    book_title = books[books.title.str.contains(sentence, case=False)].title.tolist()
    for idx, bookTitle in enumerate(book_title):
        info = books[books.title == bookTitle]
        print(info.title.tolist()[0])
        print(info.name.tolist()[0])
        print(info.weighted_score.tolist()[0])

        showClusters = st.button(label='Show review clusters for this book?', key=idx)
        if showClusters:
            with clusters:
                sentences, sentence_vectors = embedSentences(bookTitle)
                findClusters(sentences, sentence_vectors, bookTitle, k=2, n_results=2)
    if links:
        good_reads_link = goodreadsURL + book_recommends[book_recommends.book_id == (reviews[reviews.index == i].book_id.tolist() [0])].for_url.tolist()[0].replace(r'\s', '\\')
        good_reads_link

In [11]:
show_books(query)

The Tournament at Gorlan (Ranger’s Apprentice: The Early Years, #1)
John Flanagan
4.41308019439314


NameError: name 'st' is not defined

In [35]:
sentences

Series([], Name: review_text, dtype: object)

In [24]:
books[books.title.str.match('children', case=False)].book_id.tolist()

[25499718, 25938417, 7949530, 16948, 29430620, 10922889, 4140, 16130324]

In [7]:
top_n_indices, book_recommends = show_recommendations('cat', reviews, books, 50)

In [8]:
book_recommends

,title,name,description,weighted_score,book_id,for_url
0,"Saga, Vol. 2 (Saga, #2)",Brian K. Vaughan,From award-winning writer Brian K. Vaughan (Pr...,4.559449,17131869,"17131869.Saga, Vol. 2"
1,"The Darkest Hour (Warriors, #6)",Erin Hunter,"ThunderClan's darkest hour is upon them, as Ti...",4.518915,49006,49006.The Darkest Hour
2,Pete the Cat: Rocking in My School Shoes,Eric Litwin,Time to head back to school with this bestsell...,4.317648,9466024,9466024.Pete the Cat: Rocking in My School Shoes
3,"Into the Wild (Warriors, #1)",Erin Hunter,"For generations, four Clans of wild cats have ...",4.289629,111332,111332.Into the Wild
4,"This Side of the Grave (Night Huntress, #5)",Jeaniene Frost,Danger waits on both sides of the grave.\nWith...,4.279609,6871617,6871617.This Side of the Grave
5,"Saga, Vol. 1 (Saga, #1)",Brian K. Vaughan,When two soldiers from opposite sides of a nev...,4.239886,15704307,"15704307.Saga, Vol. 1"
6,Calling on Dragons (Enchanted Forest Chronicle...,Patricia C. Wrede,Those wicked wizards are back--and they've bec...,4.189605,169879,169879.Calling on Dragons
7,Where is the Green Sheep?,Mem Fox,"Here is the blue sheep, and here is the red sh...",4.187512,378164,378164.Where is the Green Sheep?
8,"Halfway to the Grave (Night Huntress, #1)",Jeaniene Frost,Flirting with the Grave...Half-vampire Catheri...,4.179877,1421990,1421990.Halfway to the Grave
9,Homer's Odyssey,Gwen Cooper,"Once in nine lives, something extraordinary ha...",4.178691,6261477,6261477.Homer's Odyssey


In [13]:
books[books.book_id.isin(reviews[reviews.index.isin(top_n_indices)].book_id.tolist())]

,title,author,average_rating,ratings_count,text_reviews_count,description,shelf_1,book_id,weighted_score,name
71,"Saga, Vol. 2 (Saga, #2)",24514.0,4.56,58474.0,4591.0,From award-winning writer Brian K. Vaughan (Pr...,NaN,17131869,4.559449,Brian K. Vaughan
135,"The Darkest Hour (Warriors, #6)",27498.0,4.52,27856.0,891.0,"ThunderClan's darkest hour is upon them, as Ti...",warriors,49006,4.518915,Erin Hunter
1239,Pete the Cat: Rocking in My School Shoes,1644249.0,4.32,8567.0,540.0,Time to head back to school with this bestsell...,picture-books,9466024,4.317648,Eric Litwin
1548,"Into the Wild (Warriors, #1)",27498.0,4.29,50490.0,3954.0,"For generations, four Clans of wild cats have ...",currently-reading,111332,4.289629,Erin Hunter
1690,"This Side of the Grave (Night Huntress, #5)",669810.0,4.28,46590.0,1978.0,Danger waits on both sides of the grave.\nWith...,vampires,6871617,4.279609,Jeaniene Frost
2269,"Saga, Vol. 1 (Saga, #1)",24514.0,4.24,142640.0,8700.0,When two soldiers from opposite sides of a nev...,NaN,15704307,4.239886,Brian K. Vaughan
3264,Calling on Dragons (Enchanted Forest Chronicle...,36122.0,4.19,34766.0,538.0,Those wicked wizards are back--and they've bec...,fantasy,169879,4.189605,Patricia C. Wrede
3376,Where is the Green Sheep?,2131.0,4.19,5483.0,326.0,"Here is the blue sheep, and here is the red sh...",picture-book,378164,4.187512,Mem Fox
3432,"Halfway to the Grave (Night Huntress, #1)",669810.0,4.18,108349.0,5496.0,Flirting with the Grave...Half-vampire Catheri...,NaN,1421990,4.179877,Jeaniene Frost
3530,Homer's Odyssey,179790.0,4.18,10086.0,1207.0,"Once in nine lives, something extraordinary ha...",non-fiction,6261477,4.178691,Gwen Cooper


In [15]:
rec_df = pd.DataFrame()

for idx, i in enumerate(reviews.iloc[top_n_indices].index):
    rec_df.append(book_recommends[book_recommends.book_id == (reviews[reviews.index == i].book_id.tolist()[0])].title.tolist()[0])
    rec_df.append(book_recommends[book_recommends.book_id == (reviews[reviews.index == i].book_id.tolist()[0])].name.tolist()[0])
    rec_df.append(round(books[books.book_id.isin(reviews[reviews.index == i].book_id.tolist())].weighted_score.tolist()[0], 2))
    rec_df.append(reviews[reviews.index == i].review_text.tolist()[0])

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [93]:
res = find_books('I love hard scifi', reviews, books, 15)

In [103]:
top_n_indices = find_reviews('I love hard scifi', reviews, 15)
book_recommends = find_books('I love hard scifi', reviews, books, 15)
book_recommends['for_url'] = book_recommends['book_id'].astype(str) + '.' + book_recommends['title'].replace(r'\(.*$', '', regex = True)

for idx, i in enumerate(reviews.iloc[top_n_indices].index):
    print(idx)
    print(i)
    print('Book title:', book_recommends[book_recommends.book_id == (reviews[reviews.index == i].book_id.tolist()[0])].title.tolist()[0])
    print('Author:', book_recommends[book_recommends.book_id == (reviews[reviews.index == i].book_id.tolist()[0])].name.tolist()[0])
    print('Weighted Score:', books[books.book_id.isin(reviews[reviews.index == i].book_id.tolist())].weighted_score.tolist()[0])
    print('Similar review:', reviews[reviews.index == i].review_text.tolist()[0])
    print('Goodreads Link:', 'https://www.goodreads.com/book/show/' + book_recommends[book_recommends.book_id == (reviews[reviews.index == i].book_id.tolist()[0])].for_url.tolist()[0])
    print('\n\n')

0
327691
Book title: Ancillary Justice (Imperial Radch, #1)
Author: Ann Leckie
Weighted Score: 3.9699322483318085
Similar review: #wickedawesome I love me some epic sci-fi
Goodreads Link: https://www.goodreads.com/book/show/17333324.Ancillary Justice 



1
210875
Book title: Ready Player One
Author: Ernest Cline
Weighted Score: 4.3094569028261365
Similar review: I love SciFi but could not get into this
Goodreads Link: https://www.goodreads.com/book/show/20603758.Ready Player One



2
248678
Book title: The Iron King (The Iron Fey, #1)
Author: Julie Kagawa
Weighted Score: 3.9199984214184576
Similar review: i like a little sci fi with my fantasy
Goodreads Link: https://www.goodreads.com/book/show/6644117.The Iron King 



3
107553
Book title: The Door Into Summer
Author: Robert A. Heinlein
Weighted Score: 3.99972628465949
Similar review: One of my all time favorite sci-fi books and I'm not a sci-fi fan.
Goodreads Link: https://www.goodreads.com/book/show/348.The Door Into Summer



4
164

In [61]:
def find_url(query, revires, books, n_results=5):
    book_data = find_books(query, reviews, books, n_results)[['title', 'book_id']]
    book_data['for_url'] = book_data['book_id'].astype(str) + '.' + book_data['title'].replace(r'\(.*$', '', regex = True)
    return book_data
url =  find_url('I love hard scifi', reviews, books, 10)

In [18]:
def show_recommendations(query, reviews, books, n_results=5):
    top_n_indices = find_reviews(query, reviews, 15)
    book_recommends = find_books(query, reviews, books, 15)
    book_recommends['for_url'] = book_recommends['book_id'].astype(str) + '.' + book_recommends['title'].replace(r'\(.*$', '', regex = True)
    common_titles = []
    for idx, i in enumerate(reviews.iloc[top_n_indices].index):
        common_titles.append(book_recommends[book_recommends.book_id == (reviews[reviews.index == i].book_id.tolist()[0])].title.tolist()[0])
        print(idx)
        print(i)
        print('Book title:', book_recommends[book_recommends.book_id == (reviews[reviews.index == i].book_id.tolist()[0])].title.tolist()[0])
        print('Author:', book_recommends[book_recommends.book_id == (reviews[reviews.index == i].book_id.tolist()[0])].name.tolist()[0])
        print('Weighted Score:', books[books.book_id.isin(reviews[reviews.index == i].book_id.tolist())].weighted_score.tolist()[0])
        print('Similar review:', reviews[reviews.index == i].review_text.tolist()[0])
        print('Goodreads Link:', 'https://www.goodreads.com/book/show/' + book_recommends[book_recommends.book_id == (reviews[reviews.index == i].book_id.tolist()[0])].for_url.tolist()[0])
        print('\n\n')
    return common_titles

In [21]:
common_titles = show_recommendations('cat', reviews, books, 50)

0
386512
Book title: This Side of the Grave (Night Huntress, #5)
Author: Jeaniene Frost
Weighted Score: 4.279608603987263
Similar review: Cat is particularly badass in this one.
Goodreads Link: https://www.goodreads.com/book/show/6871617.This Side of the Grave 



1
282434
Book title: Into the Wild (Warriors, #1)
Author: Erin Hunter
Weighted Score: 4.289628682013339
Similar review: It was about cats. I love cats.
Goodreads Link: https://www.goodreads.com/book/show/111332.Into the Wild 



2
412833
Book title: How to Tell If Your Cat Is Plotting to Kill You
Author: Matthew Inman
Weighted Score: 3.9499246100014016
Similar review: Cats are awesome, especially Bob Cats!
Goodreads Link: https://www.goodreads.com/book/show/15799936.How to Tell If Your Cat Is Plotting to Kill You



3
310544
Book title: Where is the Green Sheep?
Author: Mem Fox
Weighted Score: 4.187512349310434
Similar review: Cat introduced me to the green sheep! Thanks Cat!
Goodreads Link: https://www.goodreads.com/book/sho

In [31]:
most_common = Counter(common_titles).most_common(15)
for title, rank in most_common:
    if rank > 1:
        print(f'Title: {title}, \nAppeared: {rank} times')
    

Title: How to Tell If Your Cat Is Plotting to Kill You, 
Appeared: 3 times
Title: Into the Wild (Warriors, #1), 
Appeared: 2 times
Title: I Could Pee on This: And Other Poems by Cats, 
Appeared: 2 times


In [44]:
sidetable = pd.DataFrame(most_common, index=[x + 1 for x in range(len(most_common))]).rename(columns={0:'Title', 1:'No. of appearences'})
sidetable.iloc[0]['No. of appearences']

3

In [ ]:
def common_results(most_common):
    for title, rank in most_common:
        if rank > 1:
            a

In [17]:
top_n_indices = find_reviews('I love hard scifi', reviews, 10)
url = find_url('I love hard scifi', reviews, books, 10)
common_titles = []
for i in reviews.iloc[top_n_indices].index:
    common_titles.append(book_recommends[book_recommends.book_id == (reviews[reviews.index == i].book_id.tolist()[0])].title.tolist()[0])a
    print('Book title:', res[res.book_id == (reviews[reviews.index == i].book_id.tolist()[0])].title.tolist()[0])
    print('Author:', res[res.book_id == (reviews[reviews.index == i].book_id.tolist()[0])].name.tolist()[0])
    print('Weighted Score:', books[books.book_id.isin(reviews[reviews.index == i].book_id.tolist())].weighted_score.tolist()[0])
    print('Similar review:', reviews[reviews.index == i].review_text.tolist()[0])
    print('Goodreads Link:', 'https://www.goodreads.com/book/show/' + url[url.book_id == (reviews[reviews.index == i].book_id.tolist()[0])].for_url.tolist()[0])
    print('\n\n')

NameError: name 'find_url' is not defined

In [40]:
res['for_url'] = res['book_id'].astype(str) + '.' + res['title'].replace(r'\(.*$', '', regex = True)

In [2]:
reviews_for_cluster = pd.read_csv('/media/einhard/Seagate Expansion Drive/3380_data/data/Processed/reviews_EDA.csv').drop('Unnamed: 0', axis=1)

In [3]:
reviews_for_cluster = reviews_for_cluster[['book_id', 'review_text']]

In [9]:
for idx, i in enumerate(res.book_id):
    print(res[res.book_id==i][['title','name']])

              title          name
0  Ready Player One  Ernest Cline
                             title              name
1  Ender's Game (Ender's Saga, #1)  Orson Scott Card
                     title        name
2  Wool Omnibus (Silo, #1)  Hugh Howey
                title                    name
3  Obsidian (Lux, #1)  Jennifer L. Armentrout
                            title         name
4  Hyperion (Hyperion Cantos, #1)  Dan Simmons
              title               name
5  The Dispossessed  Ursula K. Le Guin
             title                   name
6  Spin (Spin, #1)  Robert Charles Wilson
                  title                name
7  The Door Into Summer  Robert A. Heinlein
                                    title        name
8  Ancillary Justice (Imperial Radch, #1)  Ann Leckie
                              title          name
9  The Iron King (The Iron Fey, #1)  Julie Kagawa


import pandas as pd
test = pd.read_json('/media/einhard/Seagate Expansion Drive/3380_data/data/Filtered books/20603758.Ready Player One.json')

In [16]:
from collections import Counter

# Creating cluster function

In [1]:
import pandas as pd
pd.read_csv('/media/einhard/Seagate Expansion Drive/3380_data/data/Processed/reviews_for_cluster.csv')

,831635,Recommended by our pediatrician
0,27539,6 of my smartest friends rated this 4 or highe...
1,54499,A great finish to the D'Artagnan series - defi...
2,77566,Seven amazing stories. Each one you think can'...
3,361459,Highly recommended if you are curious about Ma...
4,256008,Despite having a name that sounds like its a c...
...,...,...
1392327,170210,I loved this book. The author's politics are a...
1392328,140082,I loved this book. The author's politics are a...
1392329,110494,I loved this book. The author's politics are a...
1392330,301082,I loved this book. The author's politics are a...


In [79]:
books[books.title.isin(['Children of Time'])].book_id.tolist()

[25499718]

In [4]:
def filter_reviews(reviews_df, books_df, min_review_length=30, max_review_length=90):
    reviews_df.review_text.dropna(inplace=True)
    reviews_df = reviews_df[reviews_df.book_id.isin(books_df.book_id)]
    reviews_df = reviews_df[(reviews_df.review_text.str.len() < max_review_length) & (reviews_df.review_text.str.len() > min_review_length)]

    # Remove reviews that specify rating in body of text
    reviews_df = reviews_df[~reviews_df.review_text.str.contains(r'\d\sstar', case=False, regex=True, na=False)]
    reviews_df = reviews_df[~reviews_df.review_text.str.contains(r'\d\/\d', case=False, regex=True, na=False)]
    reviews_df = reviews_df[~reviews_df.review_text.str.contains(r'\d\.\d', case=False, regex=True, na=False)]
    reviews_df = reviews_df[~reviews_df.review_text.str.contains(r'\d\S+\D', case=False, regex=True, na=False)]
    reviews_df = reviews_df[~reviews_df.review_text.str.contains(r'spoiler', case=False, regex=True, na=False)]

    # Remove reviews that contain link to other websites
    reviews_df = reviews_df[~reviews_df.review_text.str.contains(r'http\S+', case=False, regex=True, na=False)]
    reviews_df = reviews_df[~reviews_df.review_text.str.contains(r'www\S+', case=False, regex=True, na=False)]
    return reviews_df

In [11]:
reviews_for_cluster

,book_id,review_text
0,24375664,Mind blowingly cool. Best science fiction I've...
1,18245960,This is a special book. It started slow for ab...
2,6392944,I haven't read a fun mystery book in a while a...
3,28114110,"Kevin Kelly, who is a Wired co-founder, lays o..."
4,16981,Recommended by Don Katz. Avail for free in Dec...
...,...,...
9023130,40102,This was a really great book about those first...
9023131,16165851,Interesting narrative on the growth story of R...
9023132,6114128,Successful people are objects of curiosity to ...
9023133,186224,One of the finest books that captures the esse...


In [12]:
reviews_for_cluster = filter_reviews(reviews_for_cluster, books, 30, 350)

In [13]:
reviews_for_cluster

,book_id,review_text
48,831635,Recommended by our pediatrician
63,27539,6 of my smartest friends rated this 4 or highe...
67,54499,A great finish to the D'Artagnan series - defi...
85,77566,Seven amazing stories. Each one you think can'...
87,361459,Highly recommended if you are curious about Ma...
...,...,...
9023121,170210,I loved this book. The author's politics are a...
9023122,140082,I loved this book. The author's politics are a...
9023123,110494,I loved this book. The author's politics are a...
9023124,301082,I loved this book. The author's politics are a...


In [14]:
reviews_for_cluster.to_csv('/media/einhard/Seagate Expansion Drive/3380_data/data/Processed/reviews_for_cluster.csv')

In [16]:
pd.read_csv('/media/einhard/Seagate Expansion Drive/3380_data/data/Processed/reviews_for_cluster.csv').drop('Unnamed: 0', axis=1)

,book_id,review_text
0,831635,Recommended by our pediatrician
1,27539,6 of my smartest friends rated this 4 or highe...
2,54499,A great finish to the D'Artagnan series - defi...
3,77566,Seven amazing stories. Each one you think can'...
4,361459,Highly recommended if you are curious about Ma...
...,...,...
1392328,170210,I loved this book. The author's politics are a...
1392329,140082,I loved this book. The author's politics are a...
1392330,110494,I loved this book. The author's politics are a...
1392331,301082,I loved this book. The author's politics are a...


In [80]:
from sklearn.cluster import KMeans

In [150]:
titles_list = ['Children of Time']
input_reviews = reviews_for_cluster[reviews_for_cluster.book_id.isin(books[books.title.isin(titles_list)].book_id.tolist())]

In [151]:
input_books

,book_id,review_text
534038,25499718,Really liked this one. Great world building
635878,25499718,Fantastic and original science fiction.
651656,25499718,Very enjoyable book. Unexpected ending. Settin...
940315,25499718,I really enjoyed this book. I read a lot of sc...
1013949,25499718,"Rating: Wow, really really liked it! \n Genera..."
...,...,...
8140914,25499718,Read as much as I am going/willing to .... \n ...
8446223,25499718,This book was fucking amazing. Such a unique s...
8500422,25499718,"I loved it, couldn't wait to finish it. \n The..."
8539656,25499718,"A fascinating book, though a little long winde..."


In [152]:
book_title = 'Children of Time'
review_sentences = reviews_for_cluster[reviews_for_cluster.book_id.isin(books[books.title.isin([book_title])].book_id.tolist())]

In [153]:
review_sentences.count()

book_id        62
review_text    62
dtype: int64

In [186]:
def embedSentences(book_title):

    sentences = reviews_for_cluster[reviews_for_cluster.book_id.isin(books[books.title.isin([book_title])].book_id.tolist())]['review_text']
    sentence_vectors = embed(sentences)
    return sentences, sentence_vectors

def findClusters(sentences, sentence_vectors, k, n_results):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(sentence_vectors)
    clusters = pd.DataFrame()
    for i in range(k):
        centre = kmeans.cluster_centers_[i]
        ips = np.inner(centre, sentence_vectors)
        idx = pd.Series(ips).nlargest(n_results).index
        clusteredSentences = list(sentences.iloc[idx])
        clusters.append(sentences.iloc[idx])

        st.write(f'Cluster #{i+1} sentences:\n')
        st.write(*clusteredSentences, sep='\n\n')
        print('\n')
    return clusters

In [187]:
from sklearn.cluster import KMeans

In [188]:
sentences, sentence_vectors = load_sentences(book_title)

In [192]:
data = findClusters(sentences, sentence_vectors, k=8, n_results=3)

Cluster #1 sentences:

Spiders are better than people. I love it.

dnf 
 Got horrible nightmares from the chapters with the giant jumping spiders ... could not read on :(

Wheeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee!!!! 
 Need I say more?


Cluster #2 sentences:

A novel take on first contact, and evolution. A nice book, that reminded me of some really old science fiction, yet which balanced it with the new. Good stuff.

A balanced combination of generation spanning sci-fi, ideas and new takes on technology. Best book I've read in a while.

This book takes classic science fiction like Asimov and Clarke and brings it into the modern time. The ideas are both old and new, the writing is fresh and engaging, the setting is solid and convincing, and the end result is one of the better works of classic sci-fi I've read in a long time.


Cluster #3 sentences:

Great book! I really enjoyed reading it and I'll probably be thinking about it for some time after finishing it. It may ev

In [190]:
data

['I was surprised at how good this was. \n That is not something I experience very often.',
 'Pretty much blew my mind and got me thinking about everything I thought I knew...',
 'Up there with Banks and Hamilton for me. Just had to know how it all turned out!']

In [45]:
def load_sentences(book_title):
    '''
    Function to load and embed a book's sentences
    '''
    # Copy sentence column to new variable
    sentences = input_books['review_text']

    # Vectorize sentences
    sentence_vectors = embed(sentences)
    
    return sentences, sentence_vectors

def get_clusters(sentences, sentence_vectors, k, n):
    '''
    Function to extract the n most representative sentences from k clusters
    '''

    # Instantiate the model
    kmeans_model = KMeans(n_clusters=k)

    # Fit the model
    kmeans_model.fit(sentence_array)
    
    # Loop through number of clusters
    for i in range(k):

        # Define cluster centre
        centre = kmeans_model.cluster_centers_[i]

        # Calculate inner product of cluster centre and sentence vectors
        ips = np.inner(centre, sentence_vectors)

        # Find the sentences with the highest inner products
        top_indices = pd.Series(ips).nlargest(n).index
        top_sentences = list(sentences.iloc[top_indices])

In [28]:
book_sentences, book_review_vectors = load_sentences(input_books)

In [31]:
get_clusters(book_sentences, book_review_vectors, k=5, n=8)

Cluster #1 sentences:

Really liked this one. Great world building

Very enjoyable book. Unexpected ending. Setting the stage for more.

Brilliant! I wish there were sequels!

Pretty much blew my mind and got me thinking about everything I thought I knew...

A very unique and well written story.

A great story, a very quick read

I was surprised at how good this was. 
 That is not something I experience very often.

Excellent! Can't wait to read more books by this author!!


Cluster #2 sentences:

Excellent! Can't wait to read more books by this author!!

Brilliant! I wish there were sequels!

Good read, reminded me of Vernor Vinge and his Zones of thought series.

Very enjoyable book. Unexpected ending. Setting the stage for more.

Really liked this one. Great world building

Excellent novel, interesting concepts that make you think.

A very unique and well written story.

A great story, a very quick read


Cluster #3 sentences:

Excellent! Can't wait to read more books by this author

# Creating summary names

In [1]:
import gensim